<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_01/blob/main/solver/CNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
import matplotlib as plt
from scipy.interpolate import interp1d
import matplotlib.animation

# Input the csv file
path= "/content/initial_conditions.csv"
df=pd.read_csv(path, encoding = 'latin1')

array_data = df.to_numpy()
df = df.sort_values(by='Concentration (µg/m_)')
x = df['Concentration (µg/m_)'].to_numpy()
y = df['Distance (m)'].to_numpy()
f = interp1d(x, y, kind='linear')

# generate 10 points between each interval
num_between = 5
x_new = np.linspace(x[0], x[-1], (len(x) - 1) * num_between + 1)

# evaluate interpolation
y_new = f(x_new)

print("x_new:", x_new)
print("y_new:", y_new)

class AdvectionSolver:
  #Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0


    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_grid = np.arange(0, self.L + self.dx, self.dx)
        self.num_points = len(self.x_grid)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = theta_initial.copy()
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):

        return self._solve_implicit(theta_initial, boundary_theta)

solver_A = AdvectionSolver(L, dx, dt, T, U)
res_A = solver_A.compute_solution(theta_initial_A, boundary_theta=100.0)

fig, ax = plt.subplots(figsize=(4,3))

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff() # interactive off

def animate(t):
  plt.cla()
  y = res_A[t]
  plt.plot(x_grid_A, y)
  plt.xlim(0,L)
  plt.ylim(0, res_A[t].max())
  ax.set_title("Concentration of pollutant at t = " + str(dt*t) + "s", fontsize=10, verticalalignment='top')
  plt.xlabel("x", fontsize=8)
  plt.ylabel("Concentration", fontsize=8)

matplotlib.animation.FuncAnimation(fig, animate, frames=len(res_A))

KeyError: 'Concentration (µg/m_)'